In [ ]:
import panel as pn
import pandas as pd
import numpy as np

pn.extension('perspective')

The ``Perspective`` pane provides an interactive visualization component for large, real-time datasets built on the [Perspective](https://perspective.finos.org/) project.

#### Parameters:

For layout and styling related parameters see the [customization user guide](../../user_guide/Customization.ipynb).

* **``aggregates``** (dict): Aggregation spec, e.g. {x: "distinct count"}
* **``columns``** (list): List of displayed columns.
* **``computed_columns``** (list): List of computed columns, e.g. `['"x"+"y"']`
* **``column_pivots``** (list):  A list of columns to pivot by. e.g. `["x", "y"]`
* **``filters``** (list): A list of filters, e.g. `[["x", "<", 3], ["y", "contains", "abc"]]`.
* **``object``** (dict or pd.DataFrame): The plot data declared as a dictionary of arrays or a DataFrame.
* **``row_pivots``** (list): List of columns to group by, e.g. `["x", "y"]`
* **``selectable``** (bool, default=True): Whether rows are selectable
* **``sort``** (list): List of sorting specs, e.g. `[["x", "desc"]]`
* **``plugin``** (str): The name of a plugin to display the data. For example hypergrid or d3_xy_scatter.
* **``toggle_config``** (bool): Whether to show the config menu.
* **``theme``** (str): The theme of the viewer, available options include `'material'`, `'material-dark'`, `'material-dense'`, `'material-dense-dark'` and `'vaporwave'`

___

The `Perspective` pane renders columns of data specified as a dictionary of lists or arrays and pandas DataFrames:

In [ ]:
data = {'x': [1, 2, 3], 'y': [1, 2, 3]}

pn.pane.Perspective(data)

We can also set the various config settings such as choosing the `columns` to display or the `theme` from Python:

In [ ]:
df = pd.DataFrame(np.random.randn(400, 4), columns=list('ABCD')).cumsum()

perspective = pn.pane.Perspective(
    df, plugin='d3_y_line', columns=['A', 'B', 'C', 'D'], theme='material-dark',
sizing_mode='stretch_width', height=500, margin=0)

perspective

The `Perspective` pane also supports `stream` and `patch` methods allowing us to efficiently update the data. The amount of data to keep in the streaming buffer can be controlled via the `rollover` option:

In [ ]:
rollover = pn.widgets.IntInput(name='Rollover', value=500)

def stream():
    data = df.iloc[-1] + np.random.randn(4)
    perspective.stream(data, rollover.value)

cb = pn.state.add_periodic_callback(stream, 50)

pn.Row(cb.param.period, rollover)

Alternatively we can also `patch` the data:

In [ ]:
mixed_df = pd.DataFrame({'A': np.arange(10), 'B': np.random.rand(10), 'C': [f'foo{i}' for i in range(10)]})

perspective = pn.pane.Perspective(mixed_df)

perspective

The easiest way to patch the data is by supplying a dictionary as the patch value. The dictionary should have the following structure:

```python
{
    column: [
        (index: int or slice, value),
        ...
    ],
    ...
}
```
    
As an example, below we will patch the 'A' and 'C' columns. On the `'A'` column we will replace the 0th row and on the `'C'` column we replace the first two rows:

In [ ]:
perspective.patch({'A': [(0, 3)], 'C': [(slice(0, 1), 'bar')]})